# Mecanismos de atenção e transformers

Uma grande desvantagem das redes recorrentes é que todas as palavras em uma sequência têm o mesmo impacto no resultado. Isso causa um desempenho subótimo em modelos padrão de codificador-decodificador LSTM para tarefas de sequência para sequência, como Reconhecimento de Entidades Nomeadas e Tradução Automática. Na realidade, palavras específicas na sequência de entrada frequentemente têm mais impacto nos resultados sequenciais do que outras.

Considere um modelo de sequência para sequência, como tradução automática. Ele é implementado por duas redes recorrentes, onde uma rede (**codificador**) comprime a sequência de entrada em um estado oculto, e outra (**decodificador**) expande esse estado oculto no resultado traduzido. O problema com essa abordagem é que o estado final da rede tem dificuldade em lembrar o início de uma frase, causando baixa qualidade do modelo em frases longas.

**Mecanismos de Atenção** fornecem um meio de ponderar o impacto contextual de cada vetor de entrada em cada previsão de saída da RNN. Isso é implementado criando atalhos entre os estados intermediários da RNN de entrada e a RNN de saída. Dessa forma, ao gerar o símbolo de saída $y_t$, levaremos em conta todos os estados ocultos de entrada $h_i$, com diferentes coeficientes de peso $\alpha_{t,i}$.

![Imagem mostrando um modelo codificador/decodificador com uma camada de atenção aditiva](../../../../../translated_images/pt-BR/encoder-decoder-attention.7a726296894fb567.webp)
*O modelo codificador-decodificador com mecanismo de atenção aditiva em [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), citado deste [post de blog](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

A matriz de atenção $\{\alpha_{i,j}\}$ representaria o grau em que certas palavras de entrada influenciam a geração de uma determinada palavra na sequência de saída. Abaixo está um exemplo de tal matriz:

![Imagem mostrando um alinhamento de exemplo encontrado pelo RNNsearch-50, retirada de Bahdanau - arviz.org](../../../../../translated_images/pt-BR/bahdanau-fig3.09ba2d37f202a6af.webp)

*Figura retirada de [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Os mecanismos de atenção são responsáveis por grande parte do estado da arte atual ou próximo ao estado da arte em Processamento de Linguagem Natural. No entanto, adicionar atenção aumenta significativamente o número de parâmetros do modelo, o que levou a problemas de escalabilidade com RNNs. Uma restrição chave na escalabilidade das RNNs é que a natureza recorrente dos modelos torna desafiador agrupar e paralelizar o treinamento. Em uma RNN, cada elemento de uma sequência precisa ser processado em ordem sequencial, o que significa que não pode ser facilmente paralelizado.

A adoção de mecanismos de atenção combinada com essa restrição levou à criação dos modelos Transformers, que hoje são o estado da arte e amplamente utilizados, como o BERT e o OpenGPT3.

## Modelos Transformers

Em vez de encaminhar o contexto de cada previsão anterior para a próxima etapa de avaliação, **modelos transformers** utilizam **codificações posicionais** e **atenção** para capturar o contexto de uma entrada dentro de uma janela de texto fornecida. A imagem abaixo mostra como as codificações posicionais com atenção podem capturar o contexto dentro de uma janela específica.

![GIF animado mostrando como as avaliações são realizadas em modelos transformers.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Como cada posição de entrada é mapeada independentemente para cada posição de saída, os transformers conseguem paralelizar melhor do que as RNNs, o que permite modelos de linguagem muito maiores e mais expressivos. Cada cabeça de atenção pode ser usada para aprender diferentes relações entre palavras, melhorando tarefas de Processamento de Linguagem Natural.

## Construindo um Modelo Transformer Simples

O Keras não contém uma camada Transformer integrada, mas podemos construir a nossa própria. Como antes, focaremos na classificação de texto do conjunto de dados AG News, mas vale mencionar que os modelos Transformers mostram os melhores resultados em tarefas de NLP mais complexas.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Novas camadas no Keras devem herdar da classe `Layer` e implementar o método `call`. Vamos começar com a camada **Positional Embedding**. Usaremos [algum código da documentação oficial do Keras](https://keras.io/examples/nlp/text_classification_with_transformer/). Assumiremos que preenchemos todas as sequências de entrada para o comprimento `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Esta camada consiste em duas camadas `Embedding`: uma para incorporar tokens (da maneira que discutimos anteriormente) e outra para as posições dos tokens. As posições dos tokens são criadas como uma sequência de números naturais de 0 até `maxlen` usando `tf.range`, e então passadas pela camada de embedding. Os dois vetores de embedding resultantes são somados, produzindo uma representação incorporada posicionalmente da entrada com o formato `maxlen`$\times$`embed_dim`.

Agora, vamos implementar o bloco transformer. Ele receberá a saída da camada de embedding definida anteriormente:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Agora estamos prontos para definir o modelo completo do transformer:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## Modelos Transformer BERT

**BERT** (Representações de Codificadores Bidirecionais de Transformers) é uma rede transformer muito grande e multilayer, com 12 camadas para o *BERT-base* e 24 para o *BERT-large*. O modelo é inicialmente pré-treinado em um grande corpus de dados textuais (WikiPedia + livros) usando treinamento não supervisionado (prevendo palavras mascaradas em uma sentença). Durante o pré-treinamento, o modelo adquire um nível significativo de compreensão da linguagem, que pode ser aproveitado com outros conjuntos de dados por meio de ajuste fino. Esse processo é chamado de **aprendizado por transferência**.

![imagem de http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/pt-BR/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Existem muitas variações de arquiteturas Transformer, incluindo BERT, DistilBERT, BigBird, OpenGPT3 e outras, que podem ser ajustadas.

Vamos ver como podemos usar o modelo BERT pré-treinado para resolver nosso problema tradicional de classificação de sequência. Vamos aproveitar a ideia e algum código da [documentação oficial](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Para carregar modelos pré-treinados, usaremos o **Tensorflow hub**. Primeiro, vamos carregar o vetorizador específico do BERT:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

É importante que você utilize o mesmo vetorizador que foi usado para treinar a rede original. Além disso, o vetorizador BERT retorna três componentes:
* `input_word_ids`, que é uma sequência de números de tokens para a sentença de entrada
* `input_mask`, que indica qual parte da sequência contém a entrada real e qual parte é preenchimento. É semelhante à máscara produzida pela camada `Masking`
* `input_type_ids` é usado para tarefas de modelagem de linguagem e permite especificar duas sentenças de entrada em uma única sequência.

Então, podemos instanciar o extrator de características do BERT:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Então, a camada BERT retorna vários resultados úteis:
* `pooled_output` é o resultado da média de todos os tokens na sequência. Você pode vê-lo como uma representação semântica inteligente de toda a rede. Ele é equivalente à saída da camada `GlobalAveragePooling1D` no nosso modelo anterior.
* `sequence_output` é a saída da última camada transformer (corresponde à saída de `TransformerBlock` no nosso modelo acima).
* `encoder_outputs` são as saídas de todas as camadas transformer. Como carregamos um modelo BERT de 4 camadas (como você provavelmente pode deduzir pelo nome, que contém `4_H`), ele possui 4 tensores. O último é o mesmo que `sequence_output`.

Agora vamos definir o modelo de classificação de ponta a ponta. Usaremos a *definição funcional de modelo*, onde definimos a entrada do modelo e, em seguida, fornecemos uma série de expressões para calcular sua saída. Também tornaremos os pesos do modelo BERT não treináveis e treinaremos apenas o classificador final:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Apesar de haver poucos parâmetros treináveis, o processo é bastante lento, porque o extrator de características do BERT é computacionalmente pesado. Parece que não conseguimos alcançar uma precisão razoável, seja pela falta de treinamento ou pela falta de parâmetros no modelo.

Vamos tentar desbloquear os pesos do BERT e treiná-lo também. Isso exige uma taxa de aprendizado muito pequena e uma estratégia de treinamento mais cuidadosa com **warmup**, utilizando o otimizador **AdamW**. Usaremos o pacote `tf-models-official` para criar o otimizador:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Como você pode ver, o treinamento é bastante lento - mas talvez você queira experimentar e treinar o modelo por algumas épocas (5-10) para ver se consegue obter o melhor resultado em comparação com as abordagens que usamos anteriormente.

## Biblioteca Huggingface Transformers

Outra maneira muito comum (e um pouco mais simples) de usar modelos Transformer é o [pacote HuggingFace](https://github.com/huggingface/), que fornece blocos de construção simples para diferentes tarefas de PLN. Ele está disponível tanto para Tensorflow quanto para PyTorch, outro framework de redes neurais muito popular.

> **Note**: Se você não estiver interessado em ver como a biblioteca Transformers funciona - pode pular para o final deste notebook, pois não verá nada substancialmente diferente do que fizemos acima. Estaremos repetindo os mesmos passos de treinamento do modelo BERT usando uma biblioteca diferente e um modelo substancialmente maior. Assim, o processo envolve um treinamento bastante longo, então talvez você queira apenas dar uma olhada no código.

Vamos ver como nosso problema pode ser resolvido usando [Huggingface Transformers](http://huggingface.co).


A primeira coisa que precisamos fazer é escolher o modelo que iremos utilizar. Além de alguns modelos integrados, o Huggingface possui um [repositório online de modelos](https://huggingface.co/models), onde você pode encontrar muitos outros modelos pré-treinados pela comunidade. Todos esses modelos podem ser carregados e utilizados apenas fornecendo o nome do modelo. Todos os arquivos binários necessários para o modelo serão baixados automaticamente.

Em determinados momentos, você pode precisar carregar seus próprios modelos. Nesse caso, você pode especificar o diretório que contém todos os arquivos relevantes, incluindo os parâmetros para o tokenizer, o arquivo `config.json` com os parâmetros do modelo, os pesos binários, etc.

A partir do nome do modelo, podemos instanciar tanto o modelo quanto o tokenizer. Vamos começar com o tokenizer:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

O objeto `tokenizer` contém a função `encode` que pode ser usada diretamente para codificar texto:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Também podemos usar o tokenizer para codificar uma sequência de uma maneira adequada para passar ao modelo, ou seja, incluindo os campos `token_ids`, `input_mask`, etc. Também podemos especificar que queremos tensores do Tensorflow fornecendo o argumento `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

No nosso caso, usaremos o modelo BERT pré-treinado chamado `bert-base-uncased`. *Uncased* indica que o modelo não diferencia maiúsculas de minúsculas.

Ao treinar o modelo, precisamos fornecer uma sequência tokenizada como entrada, e, portanto, iremos projetar um pipeline de processamento de dados. Como `tokenizer.encode` é uma função em Python, usaremos a mesma abordagem da última unidade, chamando-a com `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Agora podemos carregar o modelo real usando o pacote `BertForSequenceClassification`. Isso garante que nosso modelo já tenha a arquitetura necessária para classificação, incluindo o classificador final. Você verá uma mensagem de aviso indicando que os pesos do classificador final não estão inicializados e que o modelo precisará de pré-treinamento - isso é perfeitamente normal, porque é exatamente o que estamos prestes a fazer!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Como você pode ver em `summary()`, o modelo contém quase 110 milhões de parâmetros! Presumivelmente, se quisermos uma tarefa de classificação simples em um conjunto de dados relativamente pequeno, não queremos treinar a camada base do BERT:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Agora estamos prontos para começar o treinamento!

> **Nota**: Treinar um modelo BERT em escala completa pode levar muito tempo! Por isso, vamos treiná-lo apenas para as primeiras 32 batches. Isso é apenas para demonstrar como o treinamento do modelo é configurado. Se você estiver interessado em tentar o treinamento em escala completa - basta remover os parâmetros `steps_per_epoch` e `validation_steps`, e prepare-se para esperar!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Se você aumentar o número de iterações, esperar o tempo necessário e treinar por várias épocas, pode esperar que a classificação com BERT nos forneça a melhor precisão! Isso acontece porque o BERT já entende muito bem a estrutura da linguagem, e só precisamos ajustar o classificador final. No entanto, como o BERT é um modelo grande, todo o processo de treinamento leva bastante tempo e exige um poder computacional significativo! (GPU, e de preferência mais de uma).

> **Note:** Em nosso exemplo, estamos utilizando um dos menores modelos BERT pré-treinados. Existem modelos maiores que provavelmente oferecerão resultados melhores.


## Conclusão

Nesta unidade, vimos arquiteturas de modelos muito recentes baseadas em **transformers**. Aplicamos esses modelos na nossa tarefa de classificação de texto, mas, da mesma forma, os modelos BERT podem ser usados para extração de entidades, resposta a perguntas e outras tarefas de PLN.

Os modelos de transformers representam o estado da arte atual em PLN e, na maioria dos casos, devem ser a primeira solução a ser experimentada ao implementar soluções personalizadas de PLN. No entanto, compreender os princípios básicos subjacentes às redes neurais recorrentes discutidos neste módulo é extremamente importante se você deseja construir modelos neurais avançados.



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte oficial. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes do uso desta tradução.
